## 初始化

In [ ]:
# 初始化
import django_setup
import nest_asyncio
nest_asyncio.apply()
# 添加cache 
# 以下要求NotebookCacheManager.py 和 cache目录都在 当前测试文件的相同目录下（在notebook目录下）
from NotebookCacheManager import NotebookCacheManager
cache = NotebookCacheManager("cache_e1")

from pprint import pprint

# cache使用示例： 
# cache.save('存储文件名', 参数)，  a_loaded = cache.load('存储文件名')， 存在pkl文件下，点开为乱码
# cache.save_json, cache.load_json, 存在json文件下，点开可读
# cache.save_md, cache.load_md，存在md文件下，点开可读
# cache.save_json('outline1/test1') 可构建二级目录

In [ ]:
# 招标文件提取内容 准备就绪
from apps.projects.models import Project, Task, TaskType
from pprint import pprint
project = Project.objects.get(project_name='test1')
task = Task.objects.get(stage__project=project, type=TaskType.OUTLINE_ANALYSIS_TASK)
stage = task.stage
pprint(project.tender_file_extraction)
cache.save_json('招标文件/tender_file_extraction', project.tender_file_extraction)

## PART-1. 招标文件 大纲分析


### 1.1 分析第一层级标题
输入： tender_file_extraction
输出： title_lists_H1， tender_file_extraction_H1


In [ ]:
# 初始化 & 输入
import django_setup
from pprint import pprint
from NotebookCacheManager import NotebookCacheManager
cache = NotebookCacheManager("cache_e1")
tender_file_extraction = cache.load_json('招标文件/tender_file_extraction')
from apps.projects.tiptap.helpers import TiptapUtils
headings = TiptapUtils.print_headings(tender_file_extraction)
print(headings)

In [15]:
# LLM 数据准备
# 输入： tender_file_extraction, 输出：model_params, task, meta, formatted_message
# 核心功能模块： TendeorOutlinesL1 里的 prepare_context 调用 TiptapUtils.extract_indexed_paragraphs
from apps.projects.services.tasks_preparation.tender_outlines_L1 import TenderOutlinesL1
# #获取LLM分析相关数据
data_cooker_l1 = TenderOutlinesL1(tender_file_extraction)
model_params_l1, task_l1, meta_l1 = data_cooker_l1.output_params()
# # 模拟的prompt, 
simulated_prompt, formatted_message = data_cooker_l1.simulate_prompt()

cache.save_json('t1-招标文件大纲分析/t1.1-H1层级标题分析/model_params_l1', model_params_l1)
cache.save_json('t1-招标文件大纲分析/t1.1-H1层级标题分析/task_l1', task_l1)
cache.save_json('t1-招标文件大纲分析/t1.1-H1层级标题分析/meta_l1', meta_l1)
cache.save_json('t1-招标文件大纲分析/t1.1-H1层级标题分析/formatted_message', formatted_message)


In [ ]:
# LLM分析
# 输入：model_params, task_l1,  输出：针对tender_file_extraction的H1层级标题列表 （json格式）
from apps.projects.services.LLM_service.LLMcontainer import LLMService
from apps.projects.services.task_service import clean_llm_JSON_output
analyzer = LLMService(model_params_l1)
# 大模型输出的是json str格式，可能带有标记噪音
result = await analyzer.process(task_l1)
# 清除标记，转为json dict格式
result_clean_parsed = clean_llm_JSON_output(result)

print(f'result类型： {type(result)}, cleaned_result类型：{type(result_clean_parsed)}')
pprint(result_clean_parsed)

cache.save_json('t1-招标文件大纲分析/t1.1-H1层级标题分析/result', result)
cache.save_json('t1-招标文件大纲分析/t1.1-H1层级标题分析/result_clean_parsed', result_clean_parsed)

In [ ]:
# 更新文档标题
# 输入： doc, title_lits, index_path_map， 输出：doc_H1
# 核心功能组件： Tiptap.helpers.TiptapUtils.update_titles_from_list()
from apps.projects.tiptap.helpers import TiptapUtils
tender_file_extraction_H1 = TiptapUtils.update_titles_from_list(
    doc=tender_file_extraction, 
    title_list=result_clean_parsed, 
    index_path_map=meta_l1["index_path_map"]
    )

cache.save_json('招标文件/tender_file_extraction_H1', tender_file_extraction_H1)
# 检查标题是否已经改到内容里去了？
from apps.projects.tiptap.helpers import TiptapUtils
headings = TiptapUtils.find_all_headings(tender_file_extraction_H1)
print(TiptapUtils.print_headings(tender_file_extraction_H1))

### 1.2 分析第二、三层级标题
输入： tender_file_extraction_H1， 输出：title_lists_H2/3, tender_file_extraction_H23


In [ ]:
# 初始化 & 输入
import django_setup
from pprint import pprint
from NotebookCacheManager import NotebookCacheManager
cache = NotebookCacheManager("cache_e1")
tender_file_extraction_H1 =cache.load_json('招标文件/tender_file_extraction_H1')
from apps.projects.tiptap.helpers import TiptapUtils
headings = TiptapUtils.print_headings(tender_file_extraction_H1)
print(headings)


In [18]:
# LLM 数据准备
# 输入： tender_file_extraction_H1, 输出：model_params, task, meta, formatted_message
# 核心功能模块： TenderOutlinesL2 里 prepare_context 调用 TiptapUtils.extract_chapters， 
from apps.projects.services.tasks_preparation.tender_outlines_L2 import TenderOutlinesL2
# #获取LLM分析相关数据
data_cooker_l2 = TenderOutlinesL2(tender_file_extraction_H1)
model_params_l2, tasks_l2, meta_l2 = data_cooker_l2.output_params()
# # 模拟的prompt, 
simulated_prompts_l2, formatted_messages_l2 = data_cooker_l2.simulate_prompt()

cache.save_json('t1-招标文件大纲分析/t1.2-H2-3层级标题分析/model_params_l2', model_params_l2)
cache.save_json('t1-招标文件大纲分析/t1.2-H2-3层级标题分析/tasks_l2', tasks_l2)
cache.save_json('t1-招标文件大纲分析/t1.2-H2-3层级标题分析/meta_l2', meta_l2)
cache.save_json('t1-招标文件大纲分析/t1.2-H2-3层级标题分析/formatted_message', formatted_messages_l2)

In [ ]:
# LLM batch分析
from apps.projects.services.LLM_service.LLMcontainer import LLMService
from apps.projects.services.task_service import process_concurrent_JSON_outputs
analyzer = LLMService(model_params_l2)
results = await analyzer.process_with_limit(tasks_l2, limit=5)

results_clean_parsed = process_concurrent_JSON_outputs(results)
print(f'result类型： {type(results)}, cleaned_result类型：{type(results_clean_parsed)}')

# 存储OUTLINE_L2 结果
cache.save_json('t1-招标文件大纲分析/t1.2-H2-3层级标题分析/results', results)
cache.save_json('t1-招标文件大纲分析/t1.2-H2-3层级标题分析/results_clean_parsed', results_clean_parsed)

In [ ]:
# LLM 结果修正
# 将所有的'level'值加1
for item in results_clean_parsed:
    if 'level' in item:
        item['level'] += 1

from pprint import pprint
pprint(results_clean_parsed)

In [ ]:
# 更新文档标题 H2-3
# 输入： doc, title_lits, index_path_map， 输出：doc_H1
# 核心功能组件： Tiptap.helpers.TiptapUtils.update_titles_from_list()
from apps.projects.tiptap.helpers import TiptapUtils
tender_file_extraction_H2 = TiptapUtils.update_titles_from_list(
    doc=tender_file_extraction_H1, 
    title_list=results_clean_parsed, 
    index_path_map=meta_l2["index_path_map"]
    )

# 检查标题是否已经改到内容里去了？
from apps.projects.tiptap.helpers import TiptapUtils
headings = TiptapUtils.find_all_headings(tender_file_extraction_H2)
from pprint import pprint
#pprint(headings)
print(TiptapUtils.print_headings(tender_file_extraction_H2))

# 存储tender_file_extraction_L2 结果
cache.save_json('招标文件/tender_file_extraction_H2', tender_file_extraction_H2)

### 1.3 添加前言目录
输入： tender_file_extraction   输出: tender_file_extraction_final

In [ ]:
# 初始化 & 输入
import django_setup
from pprint import pprint
from NotebookCacheManager import NotebookCacheManager
cache = NotebookCacheManager("cache_e1")
tender_file_extraction_H2 =cache.load_json('招标文件/tender_file_extraction_H2')
from apps.projects.tiptap.helpers import TiptapUtils
headings = TiptapUtils.print_headings(tender_file_extraction_H2)
print(headings)

In [ ]:
# 直接添加前言
from apps.projects.tiptap.helpers import TiptapUtils
tender_file_extraction_final = TiptapUtils.add_introduction_headings(tender_file_extraction_H2)
pprint(tender_file_extraction_final)
cache.save_json('招标文件/tender_file_extraction_final', tender_file_extraction_final)

In [ ]:
#打印 带前言的文档大纲框架 
from apps.projects.tiptap.helpers import TiptapUtils
enhanced_toc=TiptapUtils.print_enhanced_toc(tender_file_extraction_final)
print(enhanced_toc)

### 1.4 增强目录分析（图表）

In [ ]:
# 初始化 & 输入
import django_setup
from pprint import pprint
from NotebookCacheManager import NotebookCacheManager
cache = NotebookCacheManager("cache_e1")
tender_file_extraction_final =cache.load_json('招标文件/tender_file_extraction_final')
from apps.projects.tiptap.helpers import TiptapUtils
headings = TiptapUtils.print_headings(tender_file_extraction_final)
print(headings)

In [ ]:
# LLM 数据准备
# 输入： tender_file_extraction_final, 输出：model_params, task, meta, formatted_message
# 核心功能模块： TenderOutlinesL2 里 prepare_context 调用 TiptapUtils.extract_chapters， 
from apps.projects.services.tasks_preparation.tender_outlines_tables import TenderOutlinesTables
# #获取LLM分析相关数据
data_cooker_tb = TenderOutlinesTables(tender_file_extraction_final)
model_params_tb, tasks_tb, meta_tb = data_cooker_tb.output_params()
# # 模拟的prompt, 
simulated_prompts_tb, formatted_messages_tb = data_cooker_tb.simulate_prompt()

cache.save_json('t1-招标文件大纲分析/t1.4-增强标题分析/model_params_tb', model_params_tb)
cache.save_json('t1-招标文件大纲分析/t1.4-增强标题分析/tasks_tb', tasks_tb)
cache.save_json('t1-招标文件大纲分析/t1.4-增强标题分析/meta_tb', meta_tb)
cache.save_json('t1-招标文件大纲分析/t1.4-增强标题分析/formatted_messages_tb', formatted_messages_tb)

In [ ]:
# LLM batch分析
from apps.projects.services.LLM_service.LLMcontainer import LLMService
from apps.projects.services.task_service import process_concurrent_JSON_outputs
analyzer = LLMService(model_params_tb)
results_tb = await analyzer.process_with_limit(tasks_tb, limit=5)
pprint(results_tb)

results_tb_clean_parsed = process_concurrent_JSON_outputs(results_tb)
print(f'result类型： {type(results_tb)}, cleaned_result类型：{type(results_tb_clean_parsed)}')

# 存储OUTLINE_L2 结果
cache.save_json('t1-招标文件大纲分析/t1.4-增强标题分析/results', results_tb)
cache.save_json('t1-招标文件大纲分析/t1.4-增强标题分析/results_clean_parsed', results_tb_clean_parsed)

In [ ]:
results_tb_clean_parsed = cache.load_json('t1-招标文件大纲分析/t1.4-增强标题分析/results_clean_parsed')
meta_tb = cache.load_json('t1-招标文件大纲分析/t1.4-增强标题分析/meta_tb')
pprint(results_tb_clean_parsed)
pprint(meta_tb)

In [ ]:
# 更新文档，给节点添加caption note
from apps.projects.tiptap.helpers import TiptapUtils
tender_file_extraction_final_tb = TiptapUtils.add_captions_to_nodes(
    doc = tender_file_extraction_final, 
    captions = results_tb_clean_parsed, 
    index_path_map = meta_tb["index_path_map"]
    )
pprint(tender_file_extraction_final_tb)
cache.save_json('招标文件/tender_file_extraction_final_tb', tender_file_extraction_final_tb)

In [ ]:
#打印 增加型文档大纲框架 
from apps.projects.tiptap.helpers import TiptapUtils
enhanced_toc=TiptapUtils.print_enhanced_toc(tender_file_extraction_final_tb)
print(enhanced_toc)

## PART-2. 投标文件编辑格式和要求

#### <font color="red"> 1. 利用LLM定位相关的章节</font>
输入： tender_file_extraction_final_tb，topic_list 输出：List[relavant_parts]

In [ ]:
# 初始化 & 输入
import django_setup
from pprint import pprint
from NotebookCacheManager import NotebookCacheManager
cache = NotebookCacheManager("cache_e1")
tender_file_extraction_final_tb =cache.load_json('招标文件/tender_file_extraction_final_tb')
from apps.projects.tiptap.helpers import TiptapUtils
headings = TiptapUtils.print_enhanced_toc(tender_file_extraction_final_tb)
print(headings)

In [2]:
# topics输入
topic_list = ["投标文件组成"]

In [ ]:
# LLM 数据准备
# 输入： tender_file_extraction_final_tb, topic_list 输出：model_params, task, meta, formatted_message
# 核心功能模块： FindTopicContextBatch 里 prepare_context 调用 TiptapUtils.print_enhanced_toc(self.doc)

from apps.projects.services.tasks_preparation.find_topic_context_batch import FindTopicContextBatch
# #获取LLM分析相关数据
data_cooker_bid = FindTopicContextBatch(tender_file_extraction_final_tb, topic_list)
model_params_bid, tasks_bid, meta_bid = data_cooker_bid.output_params()
# # 模拟的prompt, 
simulated_prompts_bid, formatted_messages_bid = data_cooker_bid.simulate_prompt()

cache.save_json('t2-投标文件编辑格式和要求/t2.1-相关章节定位/model_params_bid', model_params_bid)
cache.save_json('t2-投标文件编辑格式和要求/t2.1-相关章节定位/tasks_bid', tasks_bid)
cache.save_json('t2-投标文件编辑格式和要求/t2.1-相关章节定位/meta_bid', meta_bid)
cache.save_json('t2-投标文件编辑格式和要求/t2.1-相关章节定位/formatted_messages_bid', formatted_messages_bid)

print(type(tasks_bid), len(tasks_bid))

In [ ]:
# LLM batch分析 (虽然这里实际只处理一个任务，但为了保持一致性，还是用batch)
from apps.projects.services.LLM_service.LLMcontainer import LLMService
from apps.projects.services.task_service import process_concurrent_JSON_outputs
analyzer = LLMService(model_params_bid)
results = await analyzer.process_with_limit(tasks_bid, limit=5)
results_clean_parsed = process_concurrent_JSON_outputs(results)
pprint(results_clean_parsed)

# 清洗大模型输出结果，移除可能的Markdown代码块标记 并输出json.load格式
cache.save_json('t2-投标文件编辑格式和要求/t2.1-相关章节定位/results', results)
cache.save_json('t2-投标文件编辑格式和要求/t2.1-相关章节定位/results_clean_parsed', results_clean_parsed)

#### <font color="red"> 2. 投标文件组成框架分析</font>
输入：tender_file_extraction_final_tb, List[relevant_parts]      
输出：list[]投标文件目录框架

In [ ]:
# 初始化 & 输入
import django_setup
from pprint import pprint
from NotebookCacheManager import NotebookCacheManager
cache = NotebookCacheManager("cache_e1")
tender_file_extraction_final_tb =cache.load_json('招标文件/tender_file_extraction_final_tb')
relavant_parts = cache.load_json('t2-投标文件编辑格式和要求/t2.1-相关章节定位/results_clean_parsed')
paths = [item['path'] for item in relavant_parts]
from apps.projects.tiptap.helpers import TiptapUtils
headings = TiptapUtils.print_enhanced_toc(tender_file_extraction_final_tb)
pprint(paths)

In [ ]:
# LLM 数据准备
# 输入： tender_file_extraction_final_tb, paths 输出：model_params, task, meta, formatted_message
# 核心功能模块： BidOutlines 里 prepare_context 调用 TiptapUtils.extract_content_under_heading(self.data_input, path)
from apps.projects.services.tasks_preparation.bid_outlines import BidOutlines
# #获取LLM分析相关数据
data_cooker_bid = BidOutlines(tender_file_extraction_final_tb, paths)
model_params_bid, task_bid, meta_bid = data_cooker_bid.output_params()
# # 模拟的prompt, 
simulated_prompts_bid, formatted_messages_bid = data_cooker_bid.simulate_prompt()

cache.save_json('t2-投标文件编辑格式和要求/t2.2-投标文件组成/model_params_bid', model_params_bid)
cache.save_json('t2-投标文件编辑格式和要求/t2.2-投标文件组成/tasks_bid', task_bid)
cache.save_json('t2-投标文件编辑格式和要求/t2.2-投标文件组成/meta_bid', meta_bid)
cache.save_json('t2-投标文件编辑格式和要求/t2.2-投标文件组成/formatted_messages_bid', formatted_messages_bid)

pprint(task_bid)

In [ ]:
# LLM 分析
from apps.projects.services.LLM_service.LLMcontainer import LLMService
from apps.projects.services.task_service import clean_llm_JSON_output
analyzer = LLMService(model_params_bid)
results = await analyzer.process(task_bid)
results_clean_parsed = clean_llm_JSON_output(results)
pprint(results_clean_parsed)
cache.save_json('t2-投标文件编辑格式和要求/t2.2-投标文件组成/results', results)
cache.save_json('t2-投标文件编辑格式和要求/t2.2-投标文件组成/results_clean_parsed', results_clean_parsed)

## PART-3. 投标人资格要求

#### <font color="red"> 1. 利用LLM定位相关的章节</font>
输入： tender_file_extraction_final_tb，topic_list 输出：List[relavant_parts]

In [ ]:
# 初始化 & 输入
import django_setup
from pprint import pprint
from NotebookCacheManager import NotebookCacheManager
cache = NotebookCacheManager("cache_e1")
tender_file_extraction_final_tb =cache.load_json('招标文件/tender_file_extraction_final_tb')
from apps.projects.tiptap.helpers import TiptapUtils
headings = TiptapUtils.print_enhanced_toc(tender_file_extraction_final_tb)
print(headings)

In [2]:
# topics输入
topic_list = ["投标人资格要求", "资格要求相关的评标方法", "资格要求相关的模板"]

In [ ]:
# LLM 数据准备
# 输入： tender_file_extraction_final_tb, topic_list 输出：model_params, task, meta, formatted_message
# 核心功能模块： FindTopicContextBatch 里 prepare_context 调用 TiptapUtils.print_enhanced_toc(self.doc)

from apps.projects.services.tasks_preparation.find_topics_context import FindTopicsContext
# #获取LLM分析相关数据
data_cooker_bid = FindTopicsContext(tender_file_extraction_final_tb, topic_list)
model_params_bid, tasks_bid, meta_bid = data_cooker_bid.output_params()
# # 模拟的prompt, 
simulated_prompts_bid, formatted_messages_bid = data_cooker_bid.simulate_prompt()

cache.save_json('t3-投标人资格要求/t3.1-相关章节定位/model_params_bid', model_params_bid)
cache.save_json('t3-投标人资格要求/t3.1-相关章节定位/tasks_bid', tasks_bid)
cache.save_json('t3-投标人资格要求/t3.1-相关章节定位/meta_bid', meta_bid)
cache.save_json('t3-投标人资格要求/t3.1-相关章节定位/formatted_messages_bid', formatted_messages_bid)

print(type(tasks_bid), len(tasks_bid))

In [ ]:
# LLM batch分析 (虽然这里实际只处理一个任务，但为了保持一致性，还是用batch)
from apps.projects.services.LLM_service.LLMcontainer import LLMService
from apps.projects.services.task_service import process_concurrent_JSON_outputs
analyzer = LLMService(model_params_bid)
results = await analyzer.process_with_limit([tasks_bid], limit=5)
results_clean_parsed = process_concurrent_JSON_outputs(results)
pprint(results_clean_parsed)

# 清洗大模型输出结果，移除可能的Markdown代码块标记 并输出json.load格式
cache.save_json('t3-投标人资格要求/t3.1-相关章节定位/results', results)
cache.save_json('t3-投标人资格要求/t3.1-相关章节定位/results_clean_parsed', results_clean_parsed)

#### <font color="red"> 2. 投标人资格要求分析</font>
输入：tender_file_extraction_final_tb, List[relevant_parts]      
输出：md table

In [ ]:
# 初始化 & 输入
import django_setup
from pprint import pprint
from NotebookCacheManager import NotebookCacheManager
cache = NotebookCacheManager("cache_e1")
tender_file_extraction_final_tb =cache.load_json('招标文件/tender_file_extraction_final_tb')
relavant_parts = cache.load_json('t3-投标人资格要求/t3.1-相关章节定位/results_clean_parsed')
raw_paths = [item['path'] for item in relavant_parts]
from apps.projects.services.task_service import deduplicate_paths
paths = deduplicate_paths(raw_paths)
from apps.projects.tiptap.helpers import TiptapUtils
headings = TiptapUtils.print_enhanced_toc(tender_file_extraction_final_tb)
pprint(paths)

In [ ]:
# LLM 数据准备
# 输入： tender_file_extraction_final_tb, paths 输出：model_params, task, meta, formatted_message
# 核心功能模块： BidOutlines 里 prepare_context 调用 TiptapUtils.extract_content_under_heading(self.data_input, path)
from apps.projects.services.tasks_preparation.bidder_qulification.qualification_analysis import QualificationAnalysis
# #获取LLM分析相关数据
data_cooker_bid = QualificationAnalysis(tender_file_extraction_final_tb, paths)
model_params_bid, task_bid, meta_bid = data_cooker_bid.output_params()
# # 模拟的prompt, 
simulated_prompts_bid, formatted_messages_bid = data_cooker_bid.simulate_prompt()

cache.save_json('t3-投标人资格要求/t3.2-投标人资格要求分析/model_params_bid', model_params_bid)
cache.save_json('t3-投标人资格要求/t3.2-投标人资格要求分析/tasks_bid', task_bid)
cache.save_json('t3-投标人资格要求/t3.2-投标人资格要求分析/meta_bid', meta_bid)
cache.save_json('t3-投标人资格要求/t3.2-投标人资格要求分析/formatted_messages_bid', formatted_messages_bid)

pprint(task_bid)

In [ ]:
# LLM 分析
from apps.projects.services.LLM_service.LLMcontainer import LLMService
analyzer = LLMService(model_params_bid)
results = await analyzer.process(task_bid)
cache.save_json('t3-投标人资格要求/t3.2-投标人资格要求分析/results', results)

In [6]:
# md table Display
from IPython.display import display, HTML, Markdown
display(Markdown(results))

[
    {"标题": "有效的营业执照复印件", "模板内容": "提供有效的营业执照复印件，并在复印件上加盖公章。", "已有模板": "无"},
    {"标题": "依法缴纳社会保险和纳税记录", "模板内容": "提供近半年内任意一个月的依法缴纳社会保险记录和依法纳税记录复印件，并加盖公章。", "已有模板": "无"},
    {"标题": "近三年内没有重大违法记录的声明", "模板内容": "按照附件 3 格式准备声明文件并签字盖章。", "已有模板": "附件 3"},
    {"标题": "单位负责人为同一人或存在直接控股、管理关系的其他法人信息", "模板内容": "按照附件 12 格式填写相关信息并加盖公章。", "已有模板": "附件 12"},
    {"标题": "信用中国和中国政府采购网的信用查询记录", "模板内容": "查询信用记录并打印网页截图，加盖公章。", "已有模板": "无"},
    {"标题": "有效的《食品生产许可证》或《食品经营许可证》", "模板内容": "提供有效的《食品生产许可证》或《食品经营许可证》复印件加盖公章；若为仅销售预包装食品的企业，提供包含“食品经营（仅销售预包装食品）”的有效营业执照或备案证明材料并加盖公章。", "已有模板": "无"},
    {"标题": "会计师事务所出具的财务审计报告", "模板内容": "提供会计师事务所出具的上一年度（2023 年度）财务审计报告复印件并加盖公章；若未完成 2023 年度审计，提供 2022 年度审计报告；若为 2023 年 1 月 1 日之后成立的公司，提供财务状况说明并加盖公章。", "已有模板": "无"},
    {"标题": "法律、行政法规规定的其他条件", "模板内容": "根据具体要求提供相关证明材料。", "已有模板": "无"}
]

#### <font color="red"> 3. 投标文件侧呈现</font>
输入：tender_file_extraction_final_tb, md_table, context      
输出：json 格式

In [1]:
# 初始化 & 输入
import django_setup
from pprint import pprint
from NotebookCacheManager import NotebookCacheManager
cache = NotebookCacheManager("cache_e1")
Q_Req = cache.load_json('t3-投标人资格要求/t3.2-投标人资格要求分析/results')
task_bid = cache.load_json('t3-投标人资格要求/t3.2-投标人资格要求分析/tasks_bid')
supplement = task_bid['context']
print(supplement)

## 六、合格投标人的资格要求(本项目采用资格后审)

（1）在中华人民共和国境内注册的、具有独立承担民事责任的能力；

（2）具有依法缴纳税收和社会保险的良好记录；

（3）近三年内，在经营活动中没有重大违法记录；

（4）单位负责人为同一人或者存在直接控股、管理关系的不同投标人，不得 同时参加本项目的投标；

（5）信用中国网站（www.creditchina.gov.cn）和中国政府采购网

（www.ccgp.gov.cn）查询信用记录中（查询时间为招标公告发布后至投标截止时 间期间），被列入失信被执行人、重大税收违法案件当事人名单或政府采购严重 违法失信行为记录名单的投标人，不得参加本项目投标；

（6）具有有效的《食品生产许可证》或《食品经营许可证》。若投标人为仅 销售预包装食品的企业，须具有包含“食品经营（仅销售预包装食品）”的有效 营业执照或仅销售预包装食品备案证明材料；

（7）投标人财务状况正常，须提供会计师事务所出具的上一年度（2023年度） 财务审计报告，投标人尚未完成2023年度财务审计的须提供2022年度经审计的财

务报告；若投标人为2023年1月1日之后成立的公司，须提供财务状况说明；

（8）法律、行政法规规定的其他条件；

（9）本项目不接受联合体投标。


## 附件 11 资格证明文件

**1、有效的营业执照复印件（加盖公章）**

**2、社会保险缴纳记录和依法纳税记录（近半年内任意一个月的依法缴纳社会保**

**险记录复印件并加盖公章）**

**3、提供近三年内，在经营活动中没有重大违法记录的声明（格式见附件** **3）**

**4、在“信用中国”和“中国政府采购网”网站查询的投标人信用记录的查询结**

**果（查询时间为招标公告发布后至投标截止时间期间）（网页查询结果截图** **打印并加盖公章）**

**5、有效的《食品生产许可证》或《食品经营许可证》复印件加盖公章，若投标**

**人为仅销售预包装食品的企业，须具有包含“食品经营（仅销售预包装食品）”** **的有效营业执照或仅销售预包装食品备案证明材料并加盖公章。**

**6、会计师事务所出具的上一年度（2023** **年度）财务审计报告复印件并加盖公章，**

**投标人尚未完成** **2023** **年度财务审计的须提供** **2022

In [2]:
# LLM 数据准备
# 输入： 之前分析的结果 Q_req (str), 之前分析的context作为supplement (str) 输出：model_params, task, meta, formatted_message
from apps.projects.services.tasks_preparation.bidder_qulification.bidding_preparation import BiddingPreparation
# #获取LLM分析相关数据
data_cooker_bid = BiddingPreparation(Q_Req, supplement)
model_params_bid, task_bid, meta_bid = data_cooker_bid.output_params()
# # 模拟的prompt, 
simulated_prompts_bid, formatted_messages_bid = data_cooker_bid.simulate_prompt()

cache.save_json('t3-投标人资格要求/t3.3-投标文件呈现/model_params_bid', model_params_bid)
cache.save_json('t3-投标人资格要求/t3.3-投标文件呈现/tasks_bid', task_bid)
cache.save_json('t3-投标人资格要求/t3.3-投标文件呈现/meta_bid', meta_bid)
cache.save_json('t3-投标人资格要求/t3.3-投标文件呈现/formatted_messages_bid', formatted_messages_bid)

pprint(task_bid)

{'context': '| 编号 | 要求要点 | 所需证明材料 | 指定模板 | 准备工作建议 | 我方是否满足 |\n'
            '| --- | --- | --- | --- | --- | --- |\n'
            '| 1 | **在中华人民共和国境内注册**，具有独立承担民事责任的能力 | **有效的营业执照复印件（加盖公章）** | 无 | '
            '确保营业执照有效并在复印件上加盖公章 |  |\n'
            '| 2 | 具有依法缴纳税收和社会保险的良好记录 | '
            '**近半年内任意一个月的依法缴纳社会保险记录和依法纳税记录复印件并加盖公章** | 无 | 提供近期的社保和纳税证明，并加盖公章 '
            '|  |\n'
            '| 3 | 近三年内，在经营活动中没有重大违法记录 | **近三年内没有重大违法记录的声明（格式见附件 3）** | 附件 3 | '
            '按照附件 3 格式准备声明文件并签字盖章 |  |\n'
            '| 4 | 单位负责人为同一人或者存在直接控股、管理关系的不同投标人，不得同时参加本项目的投标 | '
            '**与投标人存在“单位负责人为同一人或者存在直接控股、管理关系”的其他法人信息（按格式附件 12 填写并加盖公章）** | 附件 '
            '12 | 按照附件 12 格式填写相关信息并加盖公章 |  |\n'
            '| 5 | '
            '在“信用中国”和“中国政府采购网”网站查询信用记录中未被列入失信被执行人、重大税收违法案件当事人名单或政府采购严重违法失信行为记录名单 '
            '| **信用中国和中国政府采购网的信用查询记录网页截图打印并加盖公章** | 无 | 查询信用记录并打印网页截图，加盖公章 |  '
            '|\n'
            '| 6 | 具有有效的《食品生产许可证》或《食品经营许可证》。若为仅销售预包装食品的企业，须提供相关备案证明 | '
            '**有效的《食品生产许可证》或

In [3]:
pprint(formatted_messages_bid)

[{'content': '你是一个专业的招标文档分析助手，帮助用户分析文档的结构和内容。', 'role': 'system'},
 {'content': '\n'
             '你将执行以下任务：\n'
             '\n'
             '【任务目标】\n'
             '\n'
             '材料A是招标文件里，关于投标人资格要求的分析结果。\n'
             '\n'
             '请以材料A里的每一项，确认是否需在投标文件里提交内容。\n'
             '\n'
             '对于需要提交的内容，请提供目录标题和内容模板。\n'
             '\n'
             '如果招标文件已经提供了模板，告知用户使用已有模板。\n'
             '\n'
             '\n'
             '\n'
             '【输出格式】\n'
             '\n'
             '        \n'
             '- 只输出符合JSON格式的数据，不要添加解释、注释或 Markdown 标记。\n'
             '- 示例：\n'
             '[\n'
             '    {"标题": str, "模板内容": str, "已有模板": str}, \n'
             '    {"标题": str, "模板内容": str, "已有模板": str}\n'
             ']\n'
             '如果招标文件未提供模板，请填无。\n'
             '\n'
             '\n'
             '以下是你将使用的内容：\n'
             '\n'
             '【材料A：主要上下文】\n'
             '| 编号 | 要求要点 | 所需证明材料 | 指定模板 | 准备工作建议 | 我方是否满足 |\n'
             '| --- | --- | --

In [ ]:
# LLM 分析
from apps.projects.services.LLM_service.LLMcontainer import LLMService
from apps.projects.services.task_service import clean_llm_JSON_output
analyzer = LLMService(model_params_bid)
results = await analyzer.process(task_bid)
results_clean_parsed = clean_llm_JSON_output(results)
cache.save_json('t3-投标人资格要求/t3.3-投标文件呈现/results', results)
cache.save_json('t3-投标人资格要求/t3.3-投标文件呈现/results_clean_parsed', results_clean_parsed)
pprint(results_clean_parsed)

HTTP Request: POST https://dashscope.aliyuncs.com/compatible-mode/v1/chat/completions "HTTP/1.1 200 OK"
[
    {"标题": "有效的营业执照复印件", "模板内容": "提供有效的营业执照复印件，并在复印件上加盖公章。", "已有模板": "无"},
    {"标题": "依法缴纳社会保险和纳税记录", "模板内容": "提供近半年内任意一个月的依法缴纳社会保险记录和依法纳税记录复印件，并加盖公章。", "已有模板": "无"},
    {"标题": "近三年内没有重大违法记录的声明", "模板内容": "按照附件 3 格式准备声明文件并签字盖章。", "已有模板": "附件 3"},
    {"标题": "单位负责人为同一人或存在直接控股、管理关系的其他法人信息", "模板内容": "按照附件 12 格式填写相关信息并加盖公章。", "已有模板": "附件 12"},
    {"标题": "信用中国和中国政府采购网的信用查询记录", "模板内容": "查询信用记录并打印网页截图，加盖公章。", "已有模板": "无"},
    {"标题": "有效的《食品生产许可证》或《食品经营许可证》", "模板内容": "提供有效的《食品生产许可证》或《食品经营许可证》复印件加盖公章；若为仅销售预包装食品的企业，提供包含“食品经营（仅销售预包装食品）”的有效营业执照或备案证明材料并加盖公章。", "已有模板": "无"},
    {"标题": "会计师事务所出具的财务审计报告", "模板内容": "提供会计师事务所出具的上一年度（2023 年度）财务审计报告复印件并加盖公章；若未完成 2023 年度审计，提供 2022 年度审计报告；若为 2023 年 1 月 1 日之后成立的公司，提供财务状况说明并加盖公章。", "已有模板": "无"},
    {"标题": "法律、行政法规规定的其他条件", "模板内容": "根据具体要求提供相关证明材料。", "已有模板": "无"}
]

In [9]:
pprint(results_clean_parsed)

[{'已有模板': '无', '标题': '有效的营业执照复印件', '模板内容': '提供有效的营业执照复印件，并在复印件上加盖公章。'},
 {'已有模板': '无',
  '标题': '依法缴纳社会保险和纳税记录',
  '模板内容': '提供近半年内任意一个月的依法缴纳社会保险记录和依法纳税记录复印件，并加盖公章。'},
 {'已有模板': '附件 3', '标题': '近三年内没有重大违法记录的声明', '模板内容': '按照附件 3 格式准备声明文件并签字盖章。'},
 {'已有模板': '附件 12',
  '标题': '单位负责人为同一人或存在直接控股、管理关系的其他法人信息',
  '模板内容': '按照附件 12 格式填写相关信息并加盖公章。'},
 {'已有模板': '无', '标题': '信用中国和中国政府采购网的信用查询记录', '模板内容': '查询信用记录并打印网页截图，加盖公章。'},
 {'已有模板': '无',
  '标题': '有效的《食品生产许可证》或《食品经营许可证》',
  '模板内容': '提供有效的《食品生产许可证》或《食品经营许可证》复印件加盖公章；若为仅销售预包装食品的企业，提供包含“食品经营（仅销售预包装食品）”的有效营业执照或备案证明材料并加盖公章。'},
 {'已有模板': '无',
  '标题': '会计师事务所出具的财务审计报告',
  '模板内容': '提供会计师事务所出具的上一年度（2023 年度）财务审计报告复印件并加盖公章；若未完成 2023 年度审计，提供 2022 '
          '年度审计报告；若为 2023 年 1 月 1 日之后成立的公司，提供财务状况说明并加盖公章。'},
 {'已有模板': '无', '标题': '法律、行政法规规定的其他条件', '模板内容': '根据具体要求提供相关证明材料。'}]


## 找到并使用模板

In [11]:
Q_Req_sections = cache.load_json('t3-投标人资格要求/t3.3-投标文件呈现/results_clean_parsed')
print(type(Q_Req_sections))
pprint(Q_Req_sections)

<class 'list'>
[{'已有模板': '无', '标题': '有效的营业执照复印件', '模板内容': '提供有效的营业执照复印件，并在复印件上加盖公章。'},
 {'已有模板': '无',
  '标题': '依法缴纳社会保险和纳税记录',
  '模板内容': '提供近半年内任意一个月的依法缴纳社会保险记录和依法纳税记录复印件，并加盖公章。'},
 {'已有模板': '附件 3', '标题': '近三年内没有重大违法记录的声明', '模板内容': '按照附件 3 格式准备声明文件并签字盖章。'},
 {'已有模板': '附件 12',
  '标题': '单位负责人为同一人或存在直接控股、管理关系的其他法人信息',
  '模板内容': '按照附件 12 格式填写相关信息并加盖公章。'},
 {'已有模板': '无', '标题': '信用中国和中国政府采购网的信用查询记录', '模板内容': '查询信用记录并打印网页截图，加盖公章。'},
 {'已有模板': '无',
  '标题': '有效的《食品生产许可证》或《食品经营许可证》',
  '模板内容': '提供有效的《食品生产许可证》或《食品经营许可证》复印件加盖公章；若为仅销售预包装食品的企业，提供包含“食品经营（仅销售预包装食品）”的有效营业执照或备案证明材料并加盖公章。'},
 {'已有模板': '无',
  '标题': '会计师事务所出具的财务审计报告',
  '模板内容': '提供会计师事务所出具的上一年度（2023 年度）财务审计报告复印件并加盖公章；若未完成 2023 年度审计，提供 2022 '
          '年度审计报告；若为 2023 年 1 月 1 日之后成立的公司，提供财务状况说明并加盖公章。'},
 {'已有模板': '无', '标题': '法律、行政法规规定的其他条件', '模板内容': '根据具体要求提供相关证明材料。'}]


In [13]:
secs_with_templates = [item for item in Q_Req_sections if item["已有模板"] != "无"]

In [14]:
pprint(secs_with_templates)

[{'已有模板': '附件 3', '标题': '近三年内没有重大违法记录的声明', '模板内容': '按照附件 3 格式准备声明文件并签字盖章。'},
 {'已有模板': '附件 12',
  '标题': '单位负责人为同一人或存在直接控股、管理关系的其他法人信息',
  '模板内容': '按照附件 12 格式填写相关信息并加盖公章。'}]


## 注意： 待 前面添加 附件的单独处理， 附件为模板的， 专门用HTML处理，保持格式。